In [1]:
# dependencies
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
from timing_diagram import time_diagram_plotter
from ShipDetectionSystemDesign import *
from spherical_earth_geometry_radar import *
from design_functions import *
from ambiguity_functions import *
from matplotlib import cm

matplotlib.use('Qt5Agg')
#%matplotlib widget
#%matplotlib notebook
%matplotlib qt

# System design Assumptions
- small antenna area 0.3 m x 2 m
- Low Peak Power ~ 200 W i.e. 1/6 Synspective
- High Duty cycle (to compensate for the low peak power) ~ 25%
- Losses + Noise Figure = 10dB (conservative)
- Resolution Area < 2m^2 (like terraSar spotlight)
- Pfa <= 10^-6 i.e. less than 1 fa for NZ EEZ
- 30 m x 7 m vessels to be detected
- LEO = 500 km


In [2]:
# antenna
la = 2  # m
wa = .3  # m
# losses + Nf
losses = 10  # dB
# orbit
h = 500e3  # m
# resolution area
ares = 2  # m2
# duty cycle
dutycycle = 0.25
# power
P_peak = 200  # W
pavg = dutycycle * P_peak  # W
# frequency
freq = 10e9
# wavelength
wavel = 299792458.0 / freq
# speed of light
c = 299792458.0

print('P_avg: ', pavg)

P_avg:  50.0


In [3]:
# probability of detection and false alarm
# expected value and variance for MEDIUM ships (150m>L>25m) 
#(from Table 1 in DLR paper) better performance are expected 
# given the higher resolution
# related to the intensity log normal distribution of ships
expected = - 0.002
variance = 4.66

# for small ships with ares , 2m2
# expected = - 0.928
# variance = 3.796

# 30 m x 7 m vessels
A_ship = 30 * 7

#Pfa = 1 * A_ship / 10 ** 7
Pfa = 10e-6

# minimum probability of detection
pd_min = 0.5


# II Analysis
sweep over a set of looking angle

In [4]:
# print(theta_mean)

In [5]:
out_dict_list = []
in_dict_list = []
theta_mean = np.array([25, 30, 35, 45, 60])
theta_mean1 = np.array([25, 35])

for theta in tqdm(theta_mean):
    out_dict, in_dict = shipDetectionSystemDesigner(theta, la, wa, ares, h, freq, P_peak, dutycycle, losses, Pfa,
                                                    A_ship, expected, variance, pd_min)
    in_dict_list.append(in_dict)
    out_dict_list.append(out_dict)
# reduced swath design
for theta in tqdm(theta_mean1):
    out_dict, in_dict = shipDetectionSystemDesigner(theta, la, wa, ares, h, freq, P_peak, dutycycle, losses, Pfa,
                                                    A_ship, expected, variance, pd_min,
                                                    reducedSwath=True, swathnominalfraction=0.7)
    in_dict_list.append(in_dict)
    out_dict_list.append(out_dict)


100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


# III Performance Evaluation - discussion
For every design, print/plot:
-	The Probability of detection over swath for different looking angles (plot)
-	NESZ min
-	The Range Ambiguity to signal Ratio over swath (plot or maximum value)
-	The Azimuth Ambiguity to Signal Ratio
-	The Required pulse bandwidth
-	The PRF
-	The swath width that effectively has a Pd higher than 0.5
-	Whether the swath is nominal or larger (3dB beam)
-	The timing Diagram (Plot)


## Plots

In [6]:
# for the plots: to be filled for every initial looking angle
# a list of broadside incidence angles for the legend
broadside_incidence_list = []
# a list of usable ground range axes
ground_range_axis_list = []
# a list of usable swath incidence angle axes
incidence_axis_list = []
# a list of probability of detection curves
pd_list = []
# a list of NESZ curves
nesz_list = []
# a list of RASR curves
rasr_list = []
# a list of AASR points
aasr_list = []
# undersampling ratios
undersampling_list = []
# a list of minimum probabilities of detection lines
pd_min_line_inc_list = []

In [7]:
# filling the lists
points = 111  # number of points per plot
for out_dict in tqdm(out_dict_list):
    # broadside incidence
    broadside_incidence = out_dict['broadside_incidence']
    broadside_incidence_list.append(broadside_incidence)

    # incidence axis
    iminmax = out_dict['usable_inc_swath']
    incidence = np.linspace(iminmax[0], iminmax[1], points)
    incidence_axis_list.append(incidence)

    # ground range axis ( from incidence )
    slant_range, ground_range = range_from_theta(incidence * 180 / np.pi, h)
    ground_range_axis_list.append(ground_range)

    # NESZ curve
    radGeo = out_dict['radarGeo']
    uniap = out_dict['uniAp']
    v_s = radGeo.abs_v
    snr_core, daz = core_snr_spherical(radGeo, uniap, incidence, wavel, v_s, h)
    B = out_dict['bandwidth']
    Ta = 300  # kelvin (antenna temperature)
    nesz = 10 ** (losses / 10) * Ta * B / (snr_core * pavg)
    nesz_list.append(nesz)

    # probability of detection curve
    P_d = pd(radGeo, uniap, incidence, wavel, losses, B, pavg, Pfa, A_ship, expected, variance)
    pd_list.append(P_d)

    # RASR curve
    #  Doppler Bandwidth
    Bd = nominal_doppler_bandwidth(uniap.L, broadside_incidence, wavel, v_s, h)
    PRI_2 = out_dict['PRI_2']
    rasr = RASR(radGeo, uniap, incidence, PRI_2, Bd, wavel, v_s, pbaroff=True)
    rasr_list.append(rasr)

    # undersampling ratio
    undersampling_list.append(Bd * PRI_2)

    # minimum probabilities of detection line
    pd_min_inc = out_dict['pd_inc_swath']
    pd_min_line_inc_list.append(pd_min_inc)

    # AASR points
    aasr = AASR(radGeo, uniap, np.average(iminmax) * 180 / np.pi, 1 / PRI_2, Bd, wavel, pbaroff=True)
    aasr_list.append(aasr)


  0%|          | 0/7 [00:00<?, ?it/s]C:\Users\smen851\PycharmProjects\design-baseline\spherical_earth_geometry_radar.py:287: RuntimeWarning: invalid value encountered in sqrt
  arg = ((lambda_c ** 2 * doppler_mesh ** 2 + np.sqrt(
100%|██████████| 7/7 [01:21<00:00, 11.68s/it]


In [8]:
# plotting the stuff

In [9]:
# pd plot
fig, ax = plt.subplots(1)
for ii in range(len(incidence_axis_list)):
    ax.plot(incidence_axis_list[ii] * 180 / np.pi, pd_list[ii], color=cm.get_cmap('tab10').colors[ii % 10])
    # min pd line
    inc_pd_min = pd_min_line_inc_list[ii]
    ax.plot(inc_pd_min * 180 / np.pi, out_dict_list[ii]['pd_min'],
            color=cm.get_cmap('tab10').colors[ii % 10],
            label=str(round(broadside_incidence_list[ii][0] * 180 / np.pi, 1)))

ax.set_xlabel('Incidence angle deg')
ax.set_ylabel('Probability of Detection')
ax.grid()
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))
plt.show()

In [10]:
# pd plot on ground range
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['svg.fonttype'] = 'none'
fig, ax = plt.subplots(1)
fig.set_size_inches(3.6, 2.5, forward=True)
for ii in range(len(incidence_axis_list)):
    ax.plot(ground_range_axis_list[ii] / 1000, pd_list[ii], color=cm.get_cmap('tab10').colors[ii % 10])
    # min pd line
    inc_pd_min = pd_min_line_inc_list[ii]
    ax.plot(out_dict_list[ii]['pd_ground_swath'] / 1000, out_dict_list[ii]['pd_min'],
            color=cm.get_cmap('tab10').colors[ii % 10],
            label='$\eta_i=$' + str(round(broadside_incidence_list[ii][0] * 180 / np.pi, 1)))

ax.set_xlabel('Ground Range [km]')
ax.set_ylabel('Probability of Detection')
ax.grid()
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))
plt.show()
plt.savefig('.\\fig.svg')
plt.close('all')

In [11]:
# nesz plot on ground range
fig, ax = plt.subplots(1)
for ii in range(len(incidence_axis_list)):
    ax.plot(ground_range_axis_list[ii] / 1000, 10 * np.log10(nesz_list[ii]),
            color=cm.get_cmap('tab10').colors[ii % 10],
            label=str(round(broadside_incidence_list[ii][0] * 180 / np.pi, 1)))

ax.set_xlabel('Ground Range [km]')
ax.set_ylabel('NESZ [dB]')
ax.grid()
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))



In [12]:
# RASR plot on ground range
fig, ax = plt.subplots(1)
for ii in range(len(incidence_axis_list)):
    ax.plot(ground_range_axis_list[ii] / 1000, 10 * np.log10(rasr_list[ii]),
            color=cm.get_cmap('tab10').colors[ii % 10],
            label=str(round(broadside_incidence_list[ii][0] * 180 / np.pi, 1)))

ax.set_xlabel('Ground Range [km]')
ax.set_ylabel('RASR [dB]')
ax.grid()
ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))

In [13]:
# RASRMAX
rasrmax = []
for ii in range(len(incidence_axis_list)):
    pd_swath = out_dict_list[ii]['pd_ground_swath']
    grax = ground_range_axis_list[ii]
    ras =rasr_list[ii]
    print(ras.shape)
    print(grax.shape)
    rasrr = np.where(float(pd_swath[1]) > grax ,
                     ras, 0)
    rasrr = np.where(grax > float(pd_swath[0]),
                     rasrr, 0)
    rasrmax.append(rasrr.max())
print(10*np.log10(rasrmax))

(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
(111,)
[-32.39266435 -18.73119058 -19.60943316 -26.82101862 -26.45712888
 -22.93750947 -19.44666469]


## Timing diagram

In [14]:
# timing diagram
plt.rcParams['svg.fonttype'] = 'none'
prf = np.linspace(250, 6000, 200)
# plotting
fig, ax = plt.subplots(1)
fig.set_size_inches(3.3, 2.5, forward=True)
time_diagram_plotter(ax, prf, dutycycle, h, integrationtime=False)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_ylim(50, 800)
ax.set_xlim(250, 6000)

# design points
for ii in range(len(incidence_axis_list)):
    swath_g = out_dict_list[ii]['ground_swath']
    PRI_2 = out_dict_list[ii]['PRI_2']
    #print(swath_g)

    pd_swath = out_dict_list[ii]['pd_ground_swath']
    #print(pd_swath)
    prff = np.abs(np.ones_like(swath_g) / PRI_2)
    ax.plot(prff, swath_g / 1000,
            color=cm.get_cmap('tab10').colors[ii % 10], linewidth=1.2,
            label=str(round(broadside_incidence_list[ii][0] * 180 / np.pi, 1)))
    ax.scatter(prff, pd_swath / 1000, marker='+', color=cm.get_cmap('tab10').colors[ii % 10])
#ax.legend(loc='center right', bbox_to_anchor=(1.11, 0.5))
plt.savefig('.\\tdfig.svg')
plt.show()
#plt.close('all')

C:\Users\smen851\PycharmProjects\design-baseline\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\smen851\PycharmProjects\design-baseline\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


In [15]:
out_dict_list[1]['pd_ground_swath']

array([300328.07548917, 361146.36183039])

## Table
AASR

In [16]:
print('eta ', '\t pd_min swath', '\t AASR', '\t\t bandwidth', '\t Bd/PRF','\t rasrmax')
for ii in range(len(out_dict_list)):
    dict = out_dict_list[ii]
    a = aasr_list[ii][0]
    uu = undersampling_list[ii][0]
    swath = dict['pd_ground_swath']
    angle = dict['broadside_incidence'] * 180 / np.pi
    print(round(angle[0], 2), '\t',
          round((swath[1] - swath[0]) / 1000, 2), 'km', '\t', round(a, 2), 'dB',
          '\t', round(dict['bandwidth'] / 1e6, 2), 'MHz', '\t', round(uu, 2),'\t', round(10*np.log10(rasrmax[ii]),2))

eta  	 pd_min swath 	 AASR 		 bandwidth 	 Bd/PRF 	 rasrmax
17.85 	 56.77 km 	 2.31 dB 	 245.33 MHz 	 2.54 	 -32.39
36.03 	 60.82 km 	 4.46 dB 	 127.73 MHz 	 3.63 	 -18.73
38.47 	 67.08 km 	 4.22 dB 	 120.77 MHz 	 4.23 	 -19.61
45.35 	 78.3 km 	 6.74 dB 	 105.59 MHz 	 6.35 	 -26.82
58.25 	 81.37 km 	 12.64 dB 	 88.32 MHz 	 12.7 	 -26.46
25.61 	 38.65 km 	 1.85 dB 	 173.9 MHz 	 1.69 	 -22.94
30.81 	 48.91 km 	 3.4 dB 	 146.72 MHz 	 2.54 	 -19.45


## Scatterplots

In [17]:
plt.show()